# Assemble Surface Records
ExaMol specifies the structure of a surface in a format which combines the surface geometry and information about where to place adsorbates.
Such data are currently stored in a directory.

In [1]:
from examol.store.recipes.surface import SurfaceRecord, SurfaceSite
from pathlib import Path
from ase.io import read
import json

## Make a Conversion Function
The directory structure holds the name of the surface, the sites are in a "site-info.json" file, and the surface slab is in a extXYZ file.

In [2]:
def convert_directory(path: Path) -> SurfaceRecord:
    """Convert a directory into a SurfaceRecord
    
    Args:
        path: Path to directory being converted
    Returns:
        Record describing the surface
    """
    
    # Start by processing the sites
    with open(path / 'site-info.json') as fp:
        site_info = json.load(fp)
    sites = [
        SurfaceSite(
            coordinate=coords,
            vector=site_info['vectors'][i]
        ) for i, coords in enumerate(site_info['coords'])
    ]
    
    # Load in the slab
    slab_path = path / 'relaxed.extxyz'
    slab = read(slab_path)
    slab_xyz = slab_path.read_text()
    
    # Make the record
    name = f'{path.parent.name}_{path.name}'
    return SurfaceRecord(
        name=name,
        slab=slab_xyz,
        surface_sites=sites,
        energy=slab.get_potential_energy()
    )

## Convert all relaxed geometries
Find all relaxed geometries and store them

In [3]:
record_paths = list(Path('surfaces/').rglob('relaxed.extxyz'))
print(f'Found {len(record_paths)} completed surfaces')

Found 2 completed surfaces


In [4]:
records = [convert_directory(p.parent) for p in record_paths]

Save the records

In [7]:
record_dir = Path('records')
record_dir.mkdir(exist_ok=True)

In [8]:
for record in records:
    with open(record_dir / f'{record.name}.json', 'w') as fp:
        print(record.json(indent=2), file=fp)